<a href="https://colab.research.google.com/github/Kun97/FIFA20_DataAnalysis/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
# Load Data
%%bash
pip install pyspark
# Download the data files from github
# If the data file does not exist in the colab environment
if [[ ! -f ./players_20.csv ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/Kun97/FIFA20_DataAnalysis/main/players_20.csv
fi

In [114]:
# The first code cell of your notebook shall include all needed imports to run your project code.  Note that
# there can be markdown cells above this cell.
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col, lit, split, isnan, when, count, isnull

In [115]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [116]:
original_data = spark.read.csv("players_20.csv", inferSchema=True, header=True)

In [117]:
# drop useless columns
drop1_columns = ['sofifa_id', 'player_url', 'long_name', 'player_tags', 'player_traits', 'real_face', 'nation_jersey_number', 'team_jersey_number', 'loaned_from', 'joined', 'contract_valid_until']
fixed1_Data = original_data.drop(*drop1_columns)

In [118]:
# drop position ability(duplicated and will affect the final result)
drop2_columns = fixed1_Data.columns[-26:]
fixed2_Data = fixed1_Data.drop(*drop2_columns)

In [119]:
# drop duplicated columns
drop3_columns = ['wage_eur', 'team_position', 'release_clause_eur', 'nation_position']
fixed3_Data = fixed2_Data.drop(*drop3_columns)

In [120]:
# split player_positions and choose the first position as primary position
split_col = split(fixed3_Data['player_positions'], ',')
position_data = fixed3_Data.withColumn('Position', split_col.getItem(0))
position_data = position_data.drop('player_positions')

In [121]:
# narrow down positions: defender
defender = ['LCB', 'RCB', 'LB', 'RB', 'CB', 'RWB', 'LWB']
defender_data = position_data.filter(col('Position').isin(defender)).withColumn('Positon_General', lit('Defender'))
defender_data.count()

5938

In [122]:
# midfileder
midfileder = ['LCM', 'LM', 'RDM', 'CAM', 'RAM', 'RCM', 'CM', 'CDM', 'RM', 'LAM', 'LDM']
midfileder_data = position_data.filter(col('Position').isin(midfileder)).withColumn('Positon_General', lit('Midfileder'))
midfileder_data.count()

6862

In [123]:
# attacker
attacker = ['ST', 'CF', 'LW', 'RW']
attacker_data = position_data.filter(col('Position').isin(attacker)).withColumn('Positon_General', lit('Attacker'))
attacker_data.count()

3442

In [124]:
# goal keeper
gk_data = position_data.filter(col('Position').isin(['GK'])).withColumn('Positon_General', lit('GK'))
gk_data.count()

2036

In [125]:
general_data = defender_data.union(midfileder_data).union(attacker_data)

In [130]:
# missing values: general data
general_data.select([count(when(isnull(c), c)).alias(c) for c in general_data.columns]).show()

+----------+---+---+---------+---------+-----------+----+-------+---------+---------+--------------+------------------------+---------+-----------+---------+---------+----+--------+-------+---------+---------+------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+------------------+--------------------+-------------------+-----------------------+--------------------+--------+---------------+
|short_name|age|dob|height_cm|weight_kg|nationality|club|overall|potential|value_eur|preferred

In [129]:
general_data = general_data.drop('gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning')

In [133]:
# missing values: gk data
gk_data.select([count(when(isnull(c), c)).alias(c) for c in gk_data.columns]).show()

+----------+---+---+---------+---------+-----------+----+-------+---------+---------+--------------+------------------------+---------+-----------+---------+---------+---------+-----------+----------+-----------+--------+--------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+------------------------+------------------+--------------------+-------------------+-----------------------+--------------------+--------+---------------+
|short_name|age|dob|height_cm|weight_kg|nationality|club|overall|potential

In [132]:
gk_data = gk_data.drop('pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic')

In [136]:
# duplicated values: general data
if general_data.count() == general_data.drop_duplicates().count():
  print('No duplicated records.')

No duplicated records.


In [137]:
# duplicated values: gk data
if gk_data.count() == gk_data.drop_duplicates().count():
  print('No duplicated records.')

No duplicated records.


In [138]:
general_data
gk_data

DataFrame[short_name: string, age: int, dob: string, height_cm: int, weight_kg: int, nationality: string, club: string, overall: int, potential: int, value_eur: int, preferred_foot: string, international_reputation: int, weak_foot: int, skill_moves: int, work_rate: string, body_type: string, gk_diving: int, gk_handling: int, gk_kicking: int, gk_reflexes: int, gk_speed: int, gk_positioning: int, attacking_crossing: int, attacking_finishing: int, attacking_heading_accuracy: int, attacking_short_passing: int, attacking_volleys: int, skill_dribbling: int, skill_curve: int, skill_fk_accuracy: int, skill_long_passing: int, skill_ball_control: int, movement_acceleration: int, movement_sprint_speed: int, movement_agility: int, movement_reactions: int, movement_balance: int, power_shot_power: int, power_jumping: int, power_stamina: int, power_strength: int, power_long_shots: int, mentality_aggression: int, mentality_interceptions: int, mentality_positioning: int, mentality_vision: int, mentalit